In [2]:
import os

import pandas as pd

In [4]:
mapping_files = [f for f in os.listdir() if f.startswith("mapping")]
mapping_files

['mapping_snv_EVE_benchmarking_20240427.csv',
 'mapping_snv_gf_EVE_benchmarking_20240427.csv',
 'mapping_benchmarking_ESM1V_20240427.csv',
 'mapping_snv_ESM1V_benchmarking_20240427.csv',
 'mapping_benchmarking_EVE_20240427.csv']

In [14]:
# Step 1: Add a S3 column to the mapping file
dfs = {}
for f in mapping_files:
    df = pd.read_csv(f)
    df["S3"] = df["file_path"].str.replace("/n/groups/marks/projects/Poly/", "")  # We'll need to copy these files as suffixes to s3://markslab-private/popEVE/
    dfs[f] = df
    df.to_csv(f, index=False)

dfs["mapping_benchmarking_EVE_20240427.csv"].head()


,protein_id,evo_model,unique_id,seen_count,not_seen_count,file_path,S3
0,NP_001341516.1,EVE,NP_001341516.1_EVE_3,876,35509,/n/groups/marks/projects/Poly/data/model_score...,data/model_score_files/popEVE_training_files_2...
1,NP_057197.2,EVE,NP_057197.2_EVE_7,460,14056,/n/groups/marks/projects/Poly/data/model_score...,data/model_score_files/popEVE_training_files_2...
2,NP_115599.2,EVE,NP_115599.2_EVE_9,1346,37300,/n/groups/marks/projects/Poly/data/model_score...,data/model_score_files/popEVE_training_files_2...
3,NP_060956.1,EVE,NP_060956.1_EVE_17,70,7093,/n/groups/marks/projects/Poly/data/model_score...,data/model_score_files/popEVE_training_files_2...
4,NP_055926.1,EVE,NP_055926.1_EVE_19,465,9871,/n/groups/marks/projects/Poly/data/model_score...,data/model_score_files/popEVE_training_files_2...


In [9]:
# Now copy all these files to s3
# Let's just get the union of all files and copy it over - extra files are okay
all_files = set()
for df in dfs.values():
    all_files = all_files.union(set(df["S3"]))
len(all_files)

33488

In [13]:
all_folders = set(("/".join(f.split("/")[:-1]) for f in all_files))
all_folders

{'data/model_score_files/popEVE_snv_gap_filter_training_files_20240427',
 'data/model_score_files/popEVE_snv_training_files_20240420',
 'data/model_score_files/popEVE_training_files_20240420'}

In [15]:
o2_prefix = "/n/groups/marks/projects/Poly/"

# Just did manually
# for folder in all_folders:
    # ! aws s3 sync {o2_prefix}/{folder} s3://markslab-private/popEVE/{folder} --recursive

^C
fatal error: 
